# Fine-tuning based on Qwen3-1.7B with Style Vector Input

In [ ]:
# Run this on Remote Jupyter Book.

import os

os.getcwd()
# os.chdir("/root/OtakuLab")

## Define Training Set Format

In [20]:
from typing_extensions import TypedDict
from typing import Optional
from pathlib import Path
import json
import re

class InstructionComponents(TypedDict):
    lexical_keywords: list[str]
    syntactic_vector: dict[str, float]
    pragmatic_styles: list[str]

class DatasetItem(TypedDict):
    character: str
    neutral_sentence: str
    instruction_components: InstructionComponents | dict[str, list[str]|dict]
    thinking_process: str
    output: str

class DatasetStorage:
    def __init__(self) -> None:
        self.items: dict[str, DatasetItem] = {}

    def __len__(self) -> int:
        return len(self.items)

    def new_item(self, character: str, neutral_sentence: str, thinking_process:str, output: str):
        item = DatasetItem(character=character,
                           neutral_sentence=neutral_sentence,
                           instruction_components={},
                           thinking_process=thinking_process,
                           output=output)
        self.items[output] = item

    def save_characters_keywords(self, character: str, lexical_keywords: list[str]):
        saved = False
        for item in self.items.values():
            if item['character'] == character:
                output = item['output']
                self.items[output]['instruction_components']['lexical_keywords'] = lexical_keywords
                saved = True
        if not saved:
            raise ValueError(f"角色 {character} 未找到")        

    def save_component(self,
                       output: str,
                       syntactic_vector: Optional[dict[str, float]] = None,
                       pragmatic_styles: Optional[list[str]] = None):
        if output not in self.items.keys():
            raise ValueError(f"风格句: {output} 似乎未加载")
        
        if syntactic_vector:
            self.items[output]['instruction_components']['syntactic_vector'] = syntactic_vector
        if pragmatic_styles:
            self.items[output]['instruction_components']['pragmatic_styles'] = pragmatic_styles

    @staticmethod
    def _verify_validity(item: DatasetItem):
        if not all((item['character'],
                    item['neutral_sentence'],
                    item['output'],
                    item['instruction_components'])):
            return False
        
        instruction_components = item['instruction_components']
        
        return all((instruction_components.get('lexical_keywords', None),
                    instruction_components.get('pragmatic_styles', None),
                    instruction_components.get("syntactic_vector", None)))
    
    @staticmethod
    def _oversampling(items_list: list[DatasetItem], item: DatasetItem):
        """
        过采样部分学习难度大的标签

        1. `tsundere` `sharp_tongued` `proud` * 5
        2. `tsukkomi` `chuunibyou` `yandere` `airhead` * 3
        """
        difficult_labels_5x = {'tsundere', 'sharp_tongued', 'proud'}
        difficult_labels_3x = {'tsukkomi', 'chuunibyou', 'yandere', 'airhead'}

        pragmatic_styles = set(item['instruction_components'].get('pragmatic_styles', []))
        
        if pragmatic_styles & difficult_labels_5x:
            for _ in range(5):
                items_list.append(item)
        elif pragmatic_styles & difficult_labels_3x:
            for _ in range(3):
                items_list.append(item)

    def output(self, output_path: Path, oversampling: bool = False):
        items = list(self.items.values())
        vaild_items = []
        vaild_items_count = 0
        
        for item in items:
            if not self._verify_validity(item):
                continue
            vaild_items.append(item)
            vaild_items_count += 1
            
            if oversampling:
                self._oversampling(vaild_items, item)

        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(vaild_items, f, ensure_ascii=False, indent=2)

        print(f"训练集已导出至 {output_path}, 总有效训练集数量: {len(vaild_items)}, 跳过数量: {len(items) - vaild_items_count}")

dataset_storage = DatasetStorage()

## Load Neutral Sentences

In [21]:
neutral_sentences_jsonl_file = Path("./data/neutral_sentences_with_CoT.jsonl")

EN_NAME_TO_ZH = {"Muice": "沐雪", "Ayaka": "神里绫华", "Zhongli": "钟离", "Hutao": "胡桃", "Haruhi": "凉宫春日"}

class NSFileItem(TypedDict):
    character: str
    original: str
    neutral: str
    CoT: str

def load_jsonl_file(jsonl_file: Path):
    with open(jsonl_file, "r", encoding="utf-8") as f:
        jsonl_file_lines = f.readlines()

    for line in jsonl_file_lines:
        if line := line.rstrip():
            item: NSFileItem = json.loads(line)
            character = EN_NAME_TO_ZH.get(item["character"], item["character"])
            neutral = item["neutral"]
            dataset_storage.new_item(character, neutral, item['CoT'], item["original"])

load_jsonl_file(neutral_sentences_jsonl_file)
print(f"成功加载了 {len(dataset_storage)} 条训练集条目")

成功加载了 6510 条训练集条目


## Process Lexical Layer Vectors

In [22]:
def get_lexical_keywords_from_file(file: Path, top_n: int = 25) -> list[str]:
    with open(file, "r", encoding="utf-8") as f:
        data: dict[str, float] = json.loads(f.read())
    return list(data.keys())[:top_n]

Lexical_Muice = get_lexical_keywords_from_file(Path("./outputs/pmi/muice_pmi_filtered.json"))
Lexical_Ayaka = get_lexical_keywords_from_file(Path("./outputs/pmi/ayaka_pmi_filtered.json"))
Lexical_Zhongli = get_lexical_keywords_from_file(Path("./outputs/pmi/zhongli_pmi_filtered.json"))
Lexical_Hutao = get_lexical_keywords_from_file(Path("./outputs/pmi/hutao_pmi_filtered.json"))
Lexical_Haruhi = get_lexical_keywords_from_file(Path("./outputs/pmi/haruhi_pmi_filtered.json"))

dataset_storage.save_characters_keywords("沐雪", Lexical_Muice)
dataset_storage.save_characters_keywords("神里绫华", Lexical_Ayaka)
dataset_storage.save_characters_keywords("钟离", Lexical_Zhongli)
dataset_storage.save_characters_keywords("胡桃", Lexical_Hutao)
dataset_storage.save_characters_keywords("凉宫春日", Lexical_Haruhi)


## Process PCFG Vectors

### Define PCFG Mapping

In [23]:
# ---- Revised PCFG-to-Dimension Mapping ----
rule_to_dim = {
    # Referentiality: 专名、指称性
    "NP -> NR": "referentiality",
    "NP -> PN": "referentiality",
    "NP -> NR NN": "referentiality",
    "DP -> DT": "referentiality",
    "NP -> NN NN": "nominal_complexity",

    # Interjectionality: 感叹与插入语气
    "INTJ -> IJ": "interjectionality",
    "FLR -> IJ": "interjectionality",
    "FLR -> SP": "interjectionality",
    "IP -> INTJ PU VP": "interjectionality",
    "IP -> INTJ VP": "interjectionality",

    # Declarativity: 陈述语气/句型层级
    "CP -> IP SP": "declarativity",
    "CP -> IP SP PU": "declarativity",
    "CP -> IP DEC": "declarativity",
    "TOP -> CP": "declarativity",
    "CP -> CP": "declarativity",
    "TOP -> IP": "declarativity",
    "VP -> VC NP": "declarativity",

    # Clausal Embedding: 子句嵌套
    "VP -> VV IP": "clausal_embedding",
    "LCP -> IP LC": "clausal_embedding",
    "IP -> ADVP PU NP VP": "clausal_embedding",
    "NP -> CP NP": "clausal_embedding",

    # Subordination: 修饰性从属结构
    "VP -> ADVP VP": "subordination",
    "IP -> NP VP": "subordination",
    "IP -> VP": "subordination",
    "VP -> VV NP": "subordination",
    "VP -> VA": "subordination",
    "IP -> VP SP": "subordination",
    "VP -> PP VP": "subordination",

    # Parallelism: 句式并列
    "VP -> VP PU VP": "parallelism",
    "VP -> VP PU VP PU VP": "parallelism",
    "UCP -> IP PU CP": "parallelism",
    "IP -> VP PU": "parallelism",
    "TOP -> UCP": "parallelism",

    # Coordination Density: 并列结构复杂度
    "NP -> NP CC NP": "coordination_density",
    "NP -> NN CC NN": "coordination_density",
    "CP -> CP CC CP": "coordination_density",

    # Modifier Density: 修饰成分密度
    "DNP -> ADJP DEG": "modifier_density",
    "DNP -> NP DEG": "modifier_density",
    "NP -> DNP NP": "modifier_density",
    "ADVP -> AD": "modifier_density",
    "ADJP -> JJ": "modifier_density",

    # Nominal Complexity: 名词短语复杂度
    "NP -> ADJP NP": "nominal_complexity",
    "NP -> DNP NP": "nominal_complexity",
    "NP -> NN NN": "nominal_complexity",

    # Prepositional Density: 介词结构使用密度
    "PP -> P LCP": "prepositional_density",
    "PP -> P NP": "prepositional_density",

    # Topic Fronting: 话题提前结构
    "TOP -> NP IP": "topic_fronting",

    # Ellipsis or Fragmentation: 口语省略、残缺句
    "IP -> VP": "ellipsis_or_fragmentation",
    "IP -> ADVP VP": "ellipsis_or_fragmentation",

    # Syntactic Compression: 句法压缩
    "NP -> NN": "syntactic_compression",
    "VP -> VV": "syntactic_compression",
    "VP -> VV VP": "syntactic_compression",

    # Quantificationality: 数量词使用
    "CLP -> M": "quantificationality",
    "QP -> CD CLP": "quantificationality",
    "NP -> DP NP": "quantificationality",
    "NP -> QP CP NP": "quantificationality",

    # Deep Embedding: 深层嵌套
    "VP -> VV NP IP": "clausal_embedding"
}


### Extract PCFG Frequency from Corpus

In [24]:
from collections import Counter, defaultdict
from typing import Any, Dict, Iterable, Tuple

class PCFGExtractor:
    def __init__(self) -> None:
        self.rules_counter: Dict[str, Counter[Tuple[str, ...]]] = defaultdict(Counter)
        self.total_rules: int = 0

    def load_trees(self, file_path: Path) -> list[dict[str, Any]]:
        with open(file_path, "r", encoding="utf-8") as f:
            return json.load(f)

    def extract_rules_from_tree(self, tree: Any) -> None:
        if not isinstance(tree, list) or len(tree) != 2:
            return
        lhs_symbol, rhs = tree
        if isinstance(rhs, list) and all(isinstance(child, list) and len(child) == 2 for child in rhs):
            rhs_symbols = tuple(child[0] for child in rhs)
            self.rules_counter[lhs_symbol][rhs_symbols] += 1
            self.total_rules += 1
            for child in rhs:
                self.extract_rules_from_tree(child)

    def extract_from_data(self, data: Iterable[dict[str, Any]]) -> None:
        for item in data:
            for tree in item.get("con", []):
                self.extract_rules_from_tree(tree)

    def build_feature_vectors(self) -> dict[str, float]:
        feature_vector: dict[str, float] = defaultdict(float)
        unmapped_rules_counter = Counter()
        unmapped_rules_count = 0

        for lhs_symbol, rhs_counter in self.rules_counter.items():
            for rhs_symbols, freq in rhs_counter.items():
                rule = f"{lhs_symbol} -> {' '.join(rhs_symbols)}"
                dim = rule_to_dim.get(rule)
                if not dim:
                    unmapped_rules_count += freq
                    unmapped_rules_counter[rule] += freq
                    continue
                feature_vector[dim] += freq

        total = sum(feature_vector.values()) or 1.0

        print(f"Total rules processed: {self.total_rules}, "
              f"Unmapped rules: {unmapped_rules_count}({unmapped_rules_count / self.total_rules * 100:.2f}%), "
              f"Mapped rules: {self.total_rules - unmapped_rules_count}({(self.total_rules - unmapped_rules_count) / self.total_rules * 100:.2f}%)\n")

        if unmapped_rules_counter:
            print("Top-5 unmapped PCFG rules:")
            for rule, freq in unmapped_rules_counter.most_common(5):
                print(f"  {rule:<50} Frequency: {freq}")
        else:
            print("All rules have been successfully mapped ✅")
        
        print()
        return {dim: freq / total for dim, freq in feature_vector.items()}

def load_and_extract(path: Path) -> dict[str, float]:
    extractor = PCFGExtractor()
    trees_data = extractor.load_trees(path)
    extractor.extract_from_data(trees_data)
    return extractor.build_feature_vectors()

def save_vector_for_character(character: str, vector: dict[str, float]) -> None:
    print(f"为角色 {character} 保存 PCFG 特征向量: {vector}")

    stored = False
    for item in dataset_storage.items.values():
        if item["character"] == character:
            dataset_storage.save_component(item["output"], syntactic_vector=vector)
            stored = True
    if not stored:
        raise ValueError(f"角色 {character} 的 PCFG 特征未匹配到任何句子")


pcfg_muice_vector = load_and_extract(Path("./outputs/cons/muice.json"))
pcfg_ayaka_vector = load_and_extract(Path("./outputs/cons/ayaka.json"))
pcfg_zhongli_vector = load_and_extract(Path("./outputs/cons/zhongli.json"))
pcfg_hutao_vector = load_and_extract(Path("./outputs/cons/hutao.json"))
pcfg_haruhi_vector = load_and_extract(Path("./outputs/cons/haruhi.json"))

save_vector_for_character("沐雪", pcfg_muice_vector)
save_vector_for_character("神里绫华", pcfg_ayaka_vector)
save_vector_for_character("钟离", pcfg_zhongli_vector)
save_vector_for_character("胡桃", pcfg_hutao_vector)
save_vector_for_character("凉宫春日", pcfg_haruhi_vector)
print("成功构建并保存 PCFG 特征向量")

Total rules processed: 70045, Unmapped rules: 16141(23.04%), Mapped rules: 53904(76.96%)

Top-5 unmapped PCFG rules:
  VP -> ADVP ADVP VP                                 Frequency: 466
  NP -> NT                                           Frequency: 412
  NP -> QP NP                                        Frequency: 408
  VP -> VP VP                                        Frequency: 346
  IP -> ADVP NP VP                                   Frequency: 283

Total rules processed: 61916, Unmapped rules: 11595(18.73%), Mapped rules: 50321(81.27%)

Top-5 unmapped PCFG rules:
  QP -> CD                                           Frequency: 394
  VP -> VE NP                                        Frequency: 375
  VP -> VV AS NP                                     Frequency: 327
  LCP -> NP LC                                       Frequency: 314
  ADVP -> CS                                         Frequency: 289

Total rules processed: 21896, Unmapped rules: 4446(20.31%), Mapped rules: 17450(79.6

## Process Pragmatic Style Layer Vectors

### Read Pragmatic Style Files for Each Corpus

In [25]:
Pragmatic_Muice = "./outputs/pragmatic/muice.jsonl"
Pragmatic_ayaka = "./outputs/pragmatic/ayaka.jsonl"
Pragmatic_zhongli = "./outputs/pragmatic/zhongli.jsonl"
Pragmatic_hutao = "./outputs/pragmatic/hutao.jsonl"
Pragmatic_haruhi = "./outputs/pragmatic/haruhi.jsonl"

class RawPCFGItem(TypedDict):
    prompt: str
    response: str
    pragmatic_styles: list[dict[str, float]]

class PCFGItem(TypedDict):
    response: str
    pragmatic_styles: list[str]

def read_pcfg_jsonl_file(jsonl_file: Path, threshold: Optional[float] = None) -> list[PCFGItem]:
    with open(jsonl_file, "r", encoding="utf-8") as f:
        lines = f.readlines()

    raw_items: list[RawPCFGItem] = []
    items: list[PCFGItem] = []

    for line in lines:
        if line := line.strip():
            raw_item: RawPCFGItem = json.loads(line)
            raw_items.append(raw_item)

    # list[dict[str, float]] -> dict[str, float] -> list[str]
    for raw_item in raw_items:
        raw_pragmatic_styles = raw_item["pragmatic_styles"]
        pragmatic_styles: dict[str, float] = {}

        for vec in raw_pragmatic_styles:
            pragmatic_styles.update(vec)

        threshold = threshold or 0
        final_styles: list[str] = []

        for key, value in pragmatic_styles.items():
            if value > threshold:
                final_styles.append(key)
        
        item = PCFGItem(response=raw_item["response"], pragmatic_styles=final_styles)
        items.append(item)

    return items

pcfg_muice_items = read_pcfg_jsonl_file(Path(Pragmatic_Muice), 0.4)
pcfg_ayaka_items = read_pcfg_jsonl_file(Path(Pragmatic_ayaka), 0.4)
pcfg_zhongli_items = read_pcfg_jsonl_file(Path(Pragmatic_zhongli), 0.4)
pcfg_hutao_items = read_pcfg_jsonl_file(Path(Pragmatic_hutao), 0.4)
pcfg_haruhi_items = read_pcfg_jsonl_file(Path(Pragmatic_haruhi), 0.4)

pcfg_import_items = pcfg_muice_items + pcfg_ayaka_items + pcfg_zhongli_items + pcfg_hutao_items + pcfg_haruhi_items

skiped = 0
for item in pcfg_import_items:
    try:
        dataset_storage.save_component(item["response"], pragmatic_styles=item["pragmatic_styles"])
    except ValueError as e:
        skiped += 1
        continue

print(f"更新了 {len(pcfg_import_items)} 条训练集条目的风格向量，已跳过: {skiped} 条")


更新了 7377 条训练集条目的风格向量，已跳过: 714 条


## Export Training Set File

In [26]:
OUTPUT_PATH = Path("./data/llm_train.json")
OVERSAMPLING_OUTPUT_PATH = Path("./data/llm_train_oversampling.json")

dataset_storage.output(OUTPUT_PATH)
dataset_storage.output(OVERSAMPLING_OUTPUT_PATH, oversampling=True)

训练集已导出至 dataset\llm_train.json, 总有效训练集数量: 5786, 跳过数量: 724
训练集已导出至 dataset\llm_train_oversampling.json, 总有效训练集数量: 6997, 跳过数量: 724


## Define Model Structure & Data Loader

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import json
from typing_extensions import TypedDict
from transformers.data.data_collator import DataCollatorWithPadding
from pathlib import Path
from transformers import Qwen3ForCausalLM
from peft import get_peft_model, LoraConfig, PeftModel

class InstructionComponents(TypedDict):
    lexical_keywords: list[str]
    syntactic_vector: dict[str, float]
    pragmatic_styles: list[str]

class DatasetItem(TypedDict):
    character: str
    neutral_sentence: str
    instruction_components: InstructionComponents
    thinking_process: str
    output: str

class StyleDataset(Dataset):
    def __init__(self, path:Path, tokenizer, prag_style_vocab: list[str], syntactic_dims: list[str], max_length=256):
        self.data: list[DatasetItem] = json.loads(path.read_text(encoding="utf-8"))
        self.tokenizer = tokenizer
        self.prag_vocab = {style: i for i, style in enumerate(prag_style_vocab)}
        self.syntactic_dims = syntactic_dims
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx:int):
        item = self.data[idx]
        instr = item["instruction_components"]

        # --- (1) 生成提示-输出对 ---
        keywords = ", ".join(instr["lexical_keywords"]) if instr["lexical_keywords"] else "None"
        pragmatic_styles = ", ".join(instr["pragmatic_styles"]) if instr["pragmatic_styles"] else "None"
        system_prompt = "You are a style transfer expert. Your task is to generate a new sentence that matches the target style, based on the content of a neutral sentence."
        user_prompt = (
            f"Target Character {item['character']}\n"
            f"Personality: {pragmatic_styles}\n"
            f"Keywords: {keywords}\n"
            f"Neutral Content: {item['neutral_sentence']}\n"
        )
        assistant_response = f"{item['thinking_process']}\n\n{item['output']}"

        # --- (2) Tokenize 完整文本 ---
        full_text = self.tokenizer.apply_chat_template(
            [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
                {"role": "assistant", "content": assistant_response}
            ],
            tokenize=False,
            add_generation_prompt=False,
            enable_thinking=False
        )

        self.tokenizer.truncation_side = "left"  # 保留结尾部分
        full_tokenized = self.tokenizer(
            full_text,
            truncation=True,
            max_length=self.max_length,
            padding=False,
        )
        input_ids = full_tokenized["input_ids"] 

        # --- (2) 构建只包含答案部分的 labels ---
        prompt_only_text = self.tokenizer.apply_chat_template(
            [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
                # 注意这里，我们使用 add_generation_prompt 来获取到 "assistant" 角色提示
                # 这是为了精确计算 prompt 的长度
            ],
            tokenize=False,
            add_generation_prompt=True
        )
        prompt_len = len(self.tokenizer(prompt_only_text).input_ids)

        labels = input_ids.copy()
        # 将 prompt 部分的 labels 设为 -100
        labels[:prompt_len] = [-100] * prompt_len

        # --- (3) 句法和语用向量 (multi-hot) ---
        syntactic_vec = torch.tensor(
            [instr["syntactic_vector"].get(dim, 0.0) for dim in self.syntactic_dims],
            dtype=torch.float
        )
        prag_vec = torch.zeros(len(self.prag_vocab), dtype=torch.float32)

        for tag in instr["pragmatic_styles"]:
            if tag in self.prag_vocab:
                prag_vec[self.prag_vocab[tag]] = 1.0

        return {
            "input_ids": input_ids,
            "syntactic_vec": syntactic_vec,
            "prag_vec": prag_vec,
            "labels": labels
        }

class StyleDataCollator(DataCollatorWithPadding):
    """动态 padding + 风格特征拼接 + 手动处理 labels"""
    def __init__(self, tokenizer, **kwargs):
        super().__init__(tokenizer=tokenizer, padding=True, **kwargs)

    def __call__(self, features):
        # 1. 从 features 中分离出风格向量和 labels
        syntactic_vecs = torch.stack([f["syntactic_vec"] for f in features])
        prag_vecs = torch.stack([f["prag_vec"] for f in features])
        labels = [f["labels"] for f in features]

        # 2. 调用父类的 collator，它现在只会处理 input_ids
        # 这会正确地 padding input_ids 并生成 attention_mask
        base_features = [
            {k: v for k, v in f.items() if k not in ("syntactic_vec", "prag_vec", "labels")}
            for f in features
        ]
        batch = super().__call__(base_features)

        # 3. 手动 padding a `labels`
        # 获取 batch 中 input_ids 被 padding 到的最大长度
        max_label_length = batch["input_ids"].shape[1]
        
        # 对每个 label 序列进行 padding，使其长度与 input_ids 一致
        padded_labels = []
        for label_seq in labels:
            # 首先，将 label 序列截断到与 input_ids 相同的最大长度
            truncated_label_seq = label_seq[:max_label_length]

            # 然后，计算基于截断后长度的 padding
            padding_length = max_label_length - len(truncated_label_seq)
            
            # 最后，添加 padding
            padded_labels.append(truncated_label_seq + [-100] * padding_length)
        
        # 将 padding 好的 labels list 转换为 tensor 并放入 batch
        batch["labels"] = torch.tensor(padded_labels, dtype=torch.long)

        # 4. 把风格特征重新放回 batch
        batch["syntactic_vec"] = syntactic_vecs
        batch["prag_vec"] = prag_vecs
        return batch

class StyleEncoder(nn.Module):
    def __init__(self, input_dim, out_dim):
        """
        input_dim: 句法向量的维度 (开始时是 8, 扩展后可能是 12-20)
        out_dim: 必须等于 `base_model.config.hidden_size` (必须与 Qwen 的 hidden_size 匹配)
        """
        super().__init__()
        # 我们可以使用一个简单的 MLP 来映射
        self.proj = nn.Sequential(
            nn.Linear(input_dim, out_dim // 2),
            nn.ReLU(),
            nn.Linear(out_dim // 2, out_dim),
            nn.Tanh() # Tanh 将输出归一化到 [-1, 1] 作为一个稳定的 "软提示"
        )

    def forward(self, syntactic_vec):
        # syntactic_vec: [batch_size, input_dim]
        # return: [batch_size, out_dim]
        return self.proj(syntactic_vec)
    
class StyleConditionedLoRAModel(nn.Module):
    def __init__(self, model_name_or_path: str|Path, syntactic_dim: int, lora_r=16, lora_alpha=16):
        super().__init__()
        
        # 1. 加载基础模型
        base_model = Qwen3ForCausalLM.from_pretrained(
            model_name_or_path,
            dtype=torch.bfloat16 # 使用 bfloat16 节省显存
        )
        
        # 2. 定义 LoRA 配置
        # 目标模块 "key/query/value"
        # 在 Qwen3-1.7B 中，它们通常被称为 "q_proj", "k_proj", "v_proj"
        config = LoraConfig(
            r=lora_r,
            lora_alpha=lora_alpha,
            target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"], # 覆盖所有线性层以获得最大表达力
            lora_dropout=0.05,
            bias="none",
            task_type="CAUSAL_LM"
        )
        
        # 3. 应用 LoRA
        self.peft_model: PeftModel = get_peft_model(base_model, config)  # type:ignore
        self.peft_model.print_trainable_parameters()
        
        # 4. 初始化 StyleEncoder
        # 它的输出必须匹配模型的隐藏层维度
        self.hidden_size = base_model.config.hidden_size
        self.style_encoder = StyleEncoder(syntactic_dim, self.hidden_size)
        # Cast the style encoder to match the base model's dtype
        self.style_encoder.to(base_model.dtype)

    def get_input_embeddings(self):
        # 获取基础模型的词嵌入层
        return self.peft_model.get_input_embeddings()  # type:ignore

    def forward(self, input_ids, attention_mask, labels, syntactic_vec):
        """
        这个 forward 函数实现了方案 C.1 (输入拼接)
        并为方案 D (辅助损失) 做好准备
        """
        
        # 1. 计算 Style Embedding
        # syntactic_vec: [batch_size, syntactic_dim]
        # style_emb: [batch_size, hidden_size]
        style_emb = self.style_encoder(syntactic_vec.to(self.peft_model.dtype)) # type: ignore
        
        # 2. 将 style_emb 视为一个 "Prefix" 软提示
        # 变为: [batch_size, 1, hidden_size]
        style_emb_prefix = style_emb.unsqueeze(1)
        
        # 3. 获取原始的 Token 词嵌入
        # token_embeds: [batch_size, seq_len, hidden_size]
        token_embeds = self.get_input_embeddings()(input_ids)
        
        # 4. 拼接 Style Prefix 和 Token 嵌入
        # inputs_embeds: [batch_size, 1 + seq_len, hidden_size]
        inputs_embeds = torch.cat([style_emb_prefix, token_embeds], dim=1)
        
        # 5. 修正 Attention Mask
        # 我们需要在 mask 的开头添加一个 "1" (代表 style prefix)
        prefix_mask = torch.ones(
            attention_mask.shape[0], 1,
            dtype=torch.long, 
            device=attention_mask.device
        )
        # new_attention_mask: [batch_size, 1 + seq_len]
        new_attention_mask = torch.cat([prefix_mask, attention_mask], dim=1)
        
        # 6. 修正 Labels
        # 我们需要 L_lm 忽略 style prefix 部分
        # 在 labels 的开头添加一个 "-100"
        prefix_labels = torch.full(
            (labels.shape[0], 1), -100, 
            dtype=torch.long, 
            device=labels.device
        )
        # new_labels: [batch_size, 1 + seq_len]
        new_labels = torch.cat([prefix_labels, labels], dim=1)
        
        # 7. 执行模型的前向传播
        # 我们请求 hidden_states 以便计算辅助损失
        outputs = self.peft_model(
            inputs_embeds=inputs_embeds,
            attention_mask=new_attention_mask,
            labels=new_labels,
            output_hidden_states=True 
        )
        
        # 8. 返回计算损失所需的所有组件
        # L_lm: outputs.loss
        # L_recon / L_style_cls: 需要 outputs.hidden_states 和 syntactic_vec/pragmatic_tags
        
        return {
            "loss": outputs.loss,  # L_lm
            "logits": outputs.logits,
            # 返回最后一层 hidden_state，用于计算辅助损失
            # hidden_states 是一个元组，最后一个元素是 [batch_size, 1 + seq_len, hidden_size]
            "last_hidden_state": outputs.hidden_states[-1] 
        }

## Load Model & Training Data

In [4]:
from transformers import AutoTokenizer
from transformers.optimization import get_scheduler
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

MODEL_PATH = "/root/autodl-tmp/Qwen3-1.7B/"
# DATASET_PATH = Path("/root/OtakuLab/dataset/llm_train.json")
DATASET_PATH = Path("/root/OtakuLab/dataset/llm_train_oversampling.json")
TEST_SIZE = 0.2

BATCH_SIZE = 6
LEARNING_RATE = 1e-5
EPOCH = 2

# 损失函数的权重
lambda_recon = 0.05  # L_recon (句法) 在前期收敛太快 
lambda_style = 0.5

# 定义风格向量信息

syntactic_dims = ['syntactic_compression', 'declarativity', 'clausal_embedding', 'nominal_complexity', 'subordination', 'interjectionality',
                  'ellipsis_or_fragmentation', 'modifier_density', 'prepositional_density', 'topic_fronting', 'referentiality', 'parallelism',
                  'quantificationality', 'coordination_density']
prag_style_vocab = ['kind', 'modest', 'clingy', 'playful', 'cold', 'proud', 'sharp_tongued', 'subservient', 'submissive', 'controlling',
                    'strong', 'defensive', 'tsukkomi', 'rational', 'curious', 'imaginative', 'cautious', 'idealistic', 'conservative',
                    'radical', 'obsessive', 'hesitant', 'energetic', 'optimistic', 'confident', 'passionate', 'melancholy', 'serious',
                    'emotional', 'sensitive', 'shy', 'irritable', 'anxious', 'lazy', 'tsundere', 'yandere', 'chuunibyou', 'cute', 'naive',
                    'airhead', 'elegant', 'humorous', 'loyal', 'responsible', 'willful', 'antisocial', 'talkative', 'masochistic', 'sadistic', 'evil']

syntactic_dim_length = len(syntactic_dims)
pragmatic_dim_length = len(prag_style_vocab)

# 加载模型和训练数据

print("Loading Model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

if tokenizer.pad_token is None:
    # Qwen 通常使用 <|endoftext|> 作为 pad token
    tokenizer.pad_token = tokenizer.eos_token 
    print(f"Set tokenizer.pad_token to {tokenizer.eos_token}")

print(f"Successfully loaded {MODEL_PATH}")

print("Loading Dataset...")
dataset = StyleDataset(DATASET_PATH, tokenizer, prag_style_vocab, syntactic_dims)
train_data, val_data = train_test_split(dataset, test_size=TEST_SIZE, random_state=42)

collator = StyleDataCollator(tokenizer, pad_to_multiple_of=8)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collator, persistent_workers=False)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collator)
print(f"Successfully loaded dataset, length: {len(dataset)}")

print("Preparing for train...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

styled_model = StyleConditionedLoRAModel(MODEL_PATH, syntactic_dim_length, lora_r=16, lora_alpha=16)

# 定义辅助损失函数
mse_loss_fn = nn.MSELoss()
bce_loss_fn = nn.BCEWithLogitsLoss() # 用于 L_style_cls

style_predictor_head = nn.Linear(styled_model.hidden_size, syntactic_dim_length + pragmatic_dim_length).to(device)
all_parameters = list(styled_model.parameters()) + list(style_predictor_head.parameters())
optimizer = torch.optim.AdamW(all_parameters, lr=LEARNING_RATE)
num_training_steps = len(train_loader) * EPOCH
lr_scheduler = get_scheduler("cosine", optimizer=optimizer, num_warmup_steps=100, num_training_steps=num_training_steps)

styled_model.to(device)
style_predictor_head.to(device)


Loading Model...
Successfully loaded /root/autodl-tmp/Qwen3-1.7B/
Loading Dataset...
Successfully loaded dataset, length: 6997
Preparing for train...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The module name  (originally ) is not a valid Python identifier. Please rename the original module to avoid import issues.


trainable params: 17,432,576 || all params: 1,738,007,552 || trainable%: 1.0030


Linear(in_features=2048, out_features=64, bias=True)

## Train Model

In [5]:
from tqdm.autonotebook import tqdm
import math

VAL_INTERVAL = 25

styled_model.train()
style_predictor_head.train()

global_step = 0
epsilon = 1e-8 

for epoch in range(EPOCH):
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCH}")
    for batch in progress_bar:
        # 将数据移动到 GPU
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        syntactic_vec = batch["syntactic_vec"].to(device)
        pragmatic_tags = batch["prag_vec"].to(device)

        # 1. 模型前向传播
        outputs = styled_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            syntactic_vec=syntactic_vec
        )
        
        # 2. 获取 L_lm (已由模型计算)
        L_lm = outputs["loss"]
        
        # --- 3. 计算辅助损失 ---
        
        # L_recon: 重建 syntactic_vec
        # 我们使用 last_hidden_state 的第一个 token (即 style prefix token) 来重建
        style_prefix_hidden_state = outputs["last_hidden_state"][:, 0] # [batch_size, hidden_size]
        
        # 通过辅助头进行预测
        aux_preds = style_predictor_head(style_prefix_hidden_state.float())
        pred_syntactic = aux_preds[:, :syntactic_dim_length]
        pred_pragmatic = aux_preds[:, syntactic_dim_length:]
        
        L_recon = mse_loss_fn(pred_syntactic, syntactic_vec)
        
        # L_style_cls: 预测 pragmatic_tags
        L_style_cls = bce_loss_fn(pred_pragmatic, pragmatic_tags.float())
        
        # 4. 计算总损失
        L_total = L_lm + lambda_recon * (L_recon / (L_recon.detach() + epsilon)) \
               + lambda_style * (L_style_cls / (L_style_cls.detach() + epsilon))
        
        # 5. 反向传播
        L_total.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        global_step += 1
        
        progress_bar.set_postfix({
                    "L_total": L_total.item(),
                    "L_lm": L_lm.item(),
                    "L_recon": L_recon.item(),
                    "L_style": L_style_cls.item(),
                    "LR": lr_scheduler.get_last_lr()[0]
                })
        
        if global_step % VAL_INTERVAL == 0:
            styled_model.eval()
            style_predictor_head.eval()

            val_loss_lm = 0
            val_loss_recon = 0
            val_loss_style = 0
            val_steps = 0

            with torch.no_grad():
                for val_batch in val_loader:
                    input_ids = val_batch["input_ids"].to(device)
                    attention_mask = val_batch["attention_mask"].to(device)
                    labels = val_batch["labels"].to(device)
                    syntactic_vec = val_batch["syntactic_vec"].to(device)
                    pragmatic_tags = val_batch["prag_vec"].to(device) # 修正了key

                    outputs = styled_model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=labels,
                        syntactic_vec=syntactic_vec
                    )     

                    L_lm_val = outputs["loss"]
                    style_prefix_hidden_state = outputs["last_hidden_state"][:, 0]
                    aux_preds = style_predictor_head(style_prefix_hidden_state.float())
                    pred_syntactic = aux_preds[:, :syntactic_dim_length]
                    pred_pragmatic = aux_preds[:, syntactic_dim_length:]

                    L_recon_val = mse_loss_fn(pred_syntactic, syntactic_vec)
                    L_style_cls_val = bce_loss_fn(pred_pragmatic, pragmatic_tags.float())

                    val_loss_lm += L_lm_val.item()
                    val_loss_recon += L_recon_val.item()
                    val_loss_style += L_style_cls_val.item()
                    val_steps += 1

            # 计算平均验证损失和 PPL
            avg_val_loss = val_loss_lm / val_steps
            ppl = math.exp(min(avg_val_loss, 20))

            print(f"Step {global_step}: Train L_total: {L_total.item():.4f}, Train L_lm {L_lm.item():.4f}, Train L_recon {L_recon.item():.4f}, Train L_style {L_style_cls.item():.4f}")
            print(f"Step {global_step}: Val PPL: {ppl:.4f}, Val L_recon: {val_loss_recon / val_steps:.4f}, Val L_style: {val_loss_style / val_steps:.4f}")

            # 切换回训练模式
            styled_model.train()
            style_predictor_head.train()

Epoch 1/2:   0%|          | 0/933 [00:00<?, ?it/s]

Step 25: Train L_total: 4.7180, Train L_lm 4.1680, Train L_recon 1.1712, Train L_style 0.7672
Step 25: Val PPL: 81.4830, Val L_recon: 1.1497, Val L_style: 0.7640
Step 50: Train L_total: 4.3247, Train L_lm 3.7747, Train L_recon 0.8670, Train L_style 0.7022
Step 50: Val PPL: 71.1566, Val L_recon: 0.8443, Val L_style: 0.6911
Step 75: Train L_total: 4.0000, Train L_lm 3.4500, Train L_recon 0.4160, Train L_style 0.5766
Step 75: Val PPL: 41.0008, Val L_recon: 0.3985, Val L_style: 0.5847
Step 100: Train L_total: 3.7942, Train L_lm 3.2442, Train L_recon 0.0738, Train L_style 0.4661
Step 100: Val PPL: 19.6680, Val L_recon: 0.0647, Val L_style: 0.4530
Step 125: Train L_total: 2.9476, Train L_lm 2.3976, Train L_recon 0.0016, Train L_style 0.3406
Step 125: Val PPL: 13.0148, Val L_recon: 0.0008, Val L_style: 0.3312
Step 150: Train L_total: 2.7074, Train L_lm 2.1574, Train L_recon 0.0003, Train L_style 0.2634
Step 150: Val PPL: 11.0672, Val L_recon: 0.0003, Val L_style: 0.2502
Step 175: Train L_tota

Epoch 2/2:   0%|          | 0/933 [00:00<?, ?it/s]

Step 950: Train L_total: 2.2939, Train L_lm 1.7439, Train L_recon 0.0002, Train L_style 0.1442
Step 950: Val PPL: 4.8566, Val L_recon: 0.0002, Val L_style: 0.1560
Step 975: Train L_total: 2.2725, Train L_lm 1.7225, Train L_recon 0.0002, Train L_style 0.1403
Step 975: Val PPL: 4.8272, Val L_recon: 0.0002, Val L_style: 0.1561
Step 1000: Train L_total: 2.0315, Train L_lm 1.4815, Train L_recon 0.0002, Train L_style 0.2136
Step 1000: Val PPL: 4.8079, Val L_recon: 0.0002, Val L_style: 0.1561
Step 1025: Train L_total: 1.8165, Train L_lm 1.2665, Train L_recon 0.0001, Train L_style 0.1356
Step 1025: Val PPL: 4.7955, Val L_recon: 0.0001, Val L_style: 0.1562
Step 1050: Train L_total: 2.1094, Train L_lm 1.5594, Train L_recon 0.0001, Train L_style 0.1913
Step 1050: Val PPL: 4.7701, Val L_recon: 0.0001, Val L_style: 0.1561
Step 1075: Train L_total: 1.8366, Train L_lm 1.2866, Train L_recon 0.0001, Train L_style 0.1666
Step 1075: Val PPL: 4.7566, Val L_recon: 0.0002, Val L_style: 0.1561
Step 1100: Tra

## Export Model

In [6]:
import os

# SAVE_DIR = Path("/root/OtakuLab/outputs/styled-qwen")
SAVE_DIR = Path("/root/OtakuLab/outputs/styled-qwen-balanced")
os.makedirs(SAVE_DIR, exist_ok=True)

print(f"Saving model to {SAVE_DIR}...")

# 1️⃣ 保存 LoRA 权重（仅包含可训练部分）
styled_model.peft_model.save_pretrained(SAVE_DIR / "lora")  # type:ignore

# 2️⃣ 保存 StyleEncoder 权重
torch.save(styled_model.style_encoder.state_dict(), SAVE_DIR / "style_encoder.pt")

# 3️⃣ 保存 StylePredictorHead 权重
torch.save(style_predictor_head.state_dict(), SAVE_DIR / "style_predictor_head.pt")

# 4️⃣ 保存 Tokenizer 配置
tokenizer.save_pretrained(SAVE_DIR / "tokenizer")

print("✅ Model checkpoint saved successfully.")


Saving model to /root/OtakuLab/outputs/styled-qwen-balanced...
✅ Model checkpoint saved successfully.


## Load Model

Before running this cell, ensure that all code in the `Define Model Structure & Data Loader` cell has been executed and model training is complete.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os
from pathlib import Path

# SAVE_DIR = Path("/root/OtakuLab/outputs/styled-qwen")
SAVE_DIR = Path("/root/OtakuLab/outputs/styled-qwen-balanced")
MODEL_PATH = Path("/root/autodl-tmp/Qwen3-1.7B/")

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

syntactic_dims = ['syntactic_compression', 'declarativity', 'clausal_embedding', 'nominal_complexity', 'subordination', 'interjectionality',
                  'ellipsis_or_fragmentation', 'modifier_density', 'prepositional_density', 'topic_fronting', 'referentiality', 'parallelism',
                  'quantificationality', 'coordination_density']
prag_style_vocab = ['kind', 'modest', 'clingy', 'playful', 'cold', 'proud', 'sharp_tongued', 'subservient', 'submissive', 'controlling',
                    'strong', 'defensive', 'tsukkomi', 'rational', 'curious', 'imaginative', 'cautious', 'idealistic', 'conservative',
                    'radical', 'obsessive', 'hesitant', 'energetic', 'optimistic', 'confident', 'passionate', 'melancholy', 'serious',
                    'emotional', 'sensitive', 'shy', 'irritable', 'anxious', 'lazy', 'tsundere', 'yandere', 'chuunibyou', 'cute', 'naive',
                    'airhead', 'elegant', 'humorous', 'loyal', 'responsible', 'willful', 'antisocial', 'talkative', 'masochistic', 'sadistic', 'evil']

syntactic_dim_length = len(syntactic_dims)

# === 加载 Tokenizer ===
tokenizer = AutoTokenizer.from_pretrained(SAVE_DIR / "tokenizer")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# === 加载基础模型并注入 LoRA ===
base_model = Qwen3ForCausalLM.from_pretrained(MODEL_PATH, dtype=torch.bfloat16)
model = PeftModel.from_pretrained(base_model, SAVE_DIR / "lora")
hidden_size = base_model.config.hidden_size

# === 初始化风格编码器 ===
style_encoder = StyleEncoder(syntactic_dim_length, hidden_size)
style_encoder.load_state_dict(torch.load(SAVE_DIR / "style_encoder.pt", map_location=DEVICE))
style_encoder.to(DEVICE)
style_encoder.eval()

model.to(DEVICE)
model.eval()
print("✅ Styled model loaded and ready for inference.")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Styled model loaded and ready for inference.


## Call Model for Inference

In [5]:
import torch

@torch.inference_mode()
def generate_styled_response(neutral_sentence: str, syntactic_vec: dict[str, float],
                             character_name: str = "Ayaka", 
                             lexical_keywords: list[str] = list(),
                             pragmatic_styles: list[str] = list(),
                             temperature: float = 0.5,
                             top_p: float = 0.9,
                             repetition_penalty: float = 1.3,
                             max_new_tokens: int = 100):
    """输入中性句和风格向量，生成风格化响应"""

    lexical_keywords = lexical_keywords or []
    pragmatic_styles = pragmatic_styles or []

    # === 1. 构建提示 ===
    keywords = ", ".join(lexical_keywords) if lexical_keywords else "None"
    pragmatics = ", ".join(pragmatic_styles) if pragmatic_styles else "None"

    system_prompt = "You are a style transfer expert. Your task is to generate a new sentence that matches the target style, based on the content of a neutral sentence."
    user_prompt = (
        f"Target Character {character_name}\n"
        f"Personality: {pragmatic_styles}\n"
        f"Keywords: {keywords}\n"
        f"Neutral Content: {neutral_sentence}\n"
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    input_text = tokenizer.apply_chat_template(messages,
                                               tokenize=False,
                                               add_generation_prompt=True,
                                               enable_thinking=True)

    # === 2. Tokenize 输入 ===
    tokenized = tokenizer(input_text, return_tensors="pt").to(DEVICE)
    input_ids = tokenized["input_ids"]
    attention_mask = tokenized["attention_mask"]

    # === 3. 风格向量 ===
    syntactic_tensor = torch.tensor(
        [syntactic_vec.get(dim, 0.0) for dim in syntactic_dims],
        dtype=torch.float32, device=DEVICE
    ).unsqueeze(0)  # [1, syntactic_dim_length]

    # === 4. 生成风格 embedding ===
    style_emb = style_encoder(syntactic_tensor).to(model.dtype)  # [1, hidden_size]
    style_prefix = style_emb.unsqueeze(1)        # [1, 1, hidden_size]

    # === 5. 获取原始词嵌入并拼接 ===
    token_embeds = model.get_input_embeddings()(input_ids)  # type:ignore
    inputs_embeds = torch.cat([style_prefix, token_embeds], dim=1)

    # === 6. Attention mask 修正 ===
    prefix_mask = torch.ones((1, 1), dtype=torch.long, device=DEVICE)
    new_attention_mask = torch.cat([prefix_mask, attention_mask], dim=1)

    # === 7. 生成 ===
    outputs = model.generate(
        inputs_embeds=inputs_embeds,
        attention_mask=new_attention_mask,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        do_sample=True
    )

    # === 8. 解码输出 ===
    # prompt_length = input_ids.shape[1]
    # new_tokens = outputs[0, prompt_length:]
    # result = tokenizer.decode(new_tokens, skip_special_tokens=True)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result


In [8]:
# --- Input for Training Example ---
neutral_sentence = "身高和体重是个人隐私，不能随便告诉你。"
styled_sentence = generate_styled_response(
    character_name="Muice",
    neutral_sentence=neutral_sentence,
    syntactic_vec={'declarativity': 0.1103257643217572, 'parallelism': 0.02918150786583556, 'ellipsis_or_fragmentation': 0.08529979222321163,
                      'subordination': 0.19688705847432472, 'interjectionality': 0.008644998515880083, 'clausal_embedding': 0.034784060552092606,
                      'referentiality': 0.11624369249035323, 'syntactic_compression': 0.18596022558622738, 'nominal_complexity': 0.03342980112793114,
                      'coordination_density': 0.002615761353517364, 'quantificationality': 0.038197536360937964, 'modifier_density': 0.1393217571979816,
                      'prepositional_density': 0.01910804392994954},
    pragmatic_styles=["cute"],
    lexical_keywords=["喵", "沐沐", "AI", "恼", "沐雪", "女孩子", "~", "⭐", "不行", "聊天", "呀", "可爱", "才", "叫", "唔", "谁", "不会", "吃", "睡觉", "笨蛋", "答", "谢谢", "把", "即", "吧，"]
)
print(f"{neutral_sentence} -> {styled_sentence}")  # Expect: 身高体重是女孩子间的秘密，怎么能轻易告诉你这种杂鱼喵！
print()

# --- Input for Example 1 ---
neutral_sentence = "我不知道这是否是个好主意。"
styled_response_2 = generate_styled_response(
    character_name="Lady Elara",
    neutral_sentence=neutral_sentence,
    syntactic_vec={
        'declarativity': 0.3, 'prepositional_density': 0.3, 'interjectionality': 0.1,
        'parallelism': 0.3, 'nominal_coordination': 0.3, 'referentiality': 0.3,
        'modifier_density': 0.7, 'subordination': 0.8
    },
    pragmatic_styles=['serious', 'rational', 'cautious', 'elegant'],
    lexical_keywords=["考虑", "荣幸", "职责", "使命", "可能", "坚定", "承认", "未来"]
)
print(f"{neutral_sentence} -> {styled_response_2}")
print()

# --- Input for Example 2 ---
neutral_sentence = "谢谢你帮我。"
styled_response_3 = generate_styled_response(
    character_name="Katsumi",
    neutral_sentence=neutral_sentence,
    syntactic_vec={
        'declarativity': 0.8, 'prepositional_density': 0.4, 'interjectionality': 0.6,
        'parallelism': 0.4, 'nominal_coordination': 0.4, 'referentiality': 0.4,
        'modifier_density': 0.4, 'subordination': 0.4
    },
    pragmatic_styles=['tsundere', 'defensive'],
    lexical_keywords=["哼", "才没有", "随便", "讨厌", "帮忙", "笨蛋", "不需要", "自己"]
)
print(f"{neutral_sentence} -> {styled_response_3}")

身高和体重是个人隐私，不能随便告诉你。 -> <think>
她用“才”和语气词~软化拒绝，自称沐雪人设可爱又带AI感。把严肃隐私话题变成撒娇式提醒，符合萌系少女的表达习惯。
</think>

身高体重是个人私密信息！你可别随便告诉别人哦

我不知道这是否是个好主意。 -> <think>
她理性而谨慎，用“可能”保留开放性。她的语言克制且优雅，“考虑是否是好主意”的措辞体现职责与使命的分寸感。
</think>

我不知道这可能是个好的决定

谢谢你帮我。 -> <think>
她用“才没有”掩饰害羞，语气强硬却暗藏软意。句尾加感叹号强化情绪，“笨蛋”和“讨厌”的反语词体现防御性与傲娇。
</think>

谢了！我才没那么好意思啊
